In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
evaluation_df = pd.read_csv('/content/drive/MyDrive/evaluation_df.csv')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import pandas as pd
from sacrebleu.metrics import BLEU

In [ ]:
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
def translate_dutch_to_croatian_batch(texts):
    tokenizer.src_lang = "nl"
    encoded_nl = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')
    generated_tokens = model.generate(**encoded_nl, forced_bos_token_id=tokenizer.get_lang_id("hr"))
    return [tokenizer.decode(g, skip_special_tokens=True) for g in generated_tokens]

def translate_croatian_to_dutch_batch(texts):
    tokenizer.src_lang = "hr"
    encoded_hr = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')
    generated_tokens = model.generate(**encoded_hr, forced_bos_token_id=tokenizer.get_lang_id("nl"))
    return [tokenizer.decode(g, skip_special_tokens=True) for g in generated_tokens]

In [ ]:
model = model.to('cuda')  # Move model to GPU

In [ ]:
from tqdm.auto import tqdm
import torch

bleu = BLEU()

# Prepare lists for translations and scores
croatian_to_dutch_translations = []
dutch_to_croatian_translations = []
croatian_to_dutch_bleu_scores = []
dutch_to_croatian_bleu_scores = []

# Convert DataFrame to batches for GPU processing
batch_size = 8  #
for i in tqdm(range(0, len(evaluation_df), batch_size)):
    batch = evaluation_df.iloc[i:i+batch_size]

    # Translate batches
    translated_dutch_batch = translate_croatian_to_dutch_batch(batch['Croatian'].tolist())  # Batch translation function
    translated_croatian_batch = translate_dutch_to_croatian_batch(batch['Dutch'].tolist())  # Batch translation function

    croatian_to_dutch_translations.extend(translated_dutch_batch)
    dutch_to_croatian_translations.extend(translated_croatian_batch)

    # Compute BLEU scores in batch
    for translated_dutch, translated_croatian, original_dutch, original_croatian in zip(translated_dutch_batch, translated_croatian_batch, batch['Dutch'].tolist(), batch['Croatian'].tolist()):
        dutch_score = bleu.corpus_score([translated_dutch], [[original_dutch]])
        croatian_score = bleu.corpus_score([translated_croatian], [[original_croatian]])

        croatian_to_dutch_bleu_scores.append(dutch_score.score)
        dutch_to_croatian_bleu_scores.append(croatian_score.score)

# Update DataFrame
evaluation_df['Translated Dutch'] = croatian_to_dutch_translations
evaluation_df['Translated Croatian'] = dutch_to_croatian_translations
evaluation_df['Croatian to Dutch BLEU'] = croatian_to_dutch_bleu_scores
evaluation_df['Dutch to Croatian BLEU'] = dutch_to_croatian_bleu_scores

  0%|          | 0/279 [00:00<?, ?it/s]

In [ ]:
evaluation_df["Croatian to Dutch BLEU"].mean()

19.61287093965467

In [ ]:
evaluation_df["Dutch to Croatian BLEU"].mean()

17.935117451122714

In [ ]:
evaluation_df.head(50)

,Unnamed: 0,Dutch,Croatian,Translated Dutch,Translated Croatian,Croatian to Dutch BLEU,Dutch to Croatian BLEU
0,0,Op maandag kondigden wetenschappers van de Sta...,U ponedjeljak su znanstvenici s Medicinskog fa...,Op maandag konden wetenschappers van de School...,U ponedjeljak su znanstvenici sa Stanford Univ...,8.856387,7.853061
1,1,Hoofdonderzoekers zeggen dat dit kan leiden to...,Voditelji istraživanja izjavili su da bi ovo o...,De leiders van het onderzoek zeiden dat deze o...,Glavni istraživači kažu da to može dovesti do ...,26.109285,25.740200
2,2,De JAS 39C Gripen stortte rond 09.30 uur lokal...,JAS 39C Gripen srušio se na pistu i eksplodira...,De JAS 39C Gripen raakte op een spoor en explo...,JAS 39C Gripen srušio se oko 09.30 sati lokaln...,15.773903,37.837561
3,3,De piloot werd geïdentificeerd als majoor Dilo...,Pilot je identificiran kao zapovjednik eskadri...,De piloot werd geïdentificeerd als commandant ...,Pilot je identificiran kao glavni Dilokrit Pat...,38.163309,44.124845
4,4,De lokale media meldt dat er tijdens een actie...,Lokalni mediji izvješćuju da je došlo do prevr...,Lokale media melden dat er een brandweer van d...,Lokalni mediji izvijestili su da je tijekom ak...,6.228497,6.219512
5,5,De 28-jaar oude Vidal is drie seizoenen gelede...,28-godišnji Vidal došao je iz Seville u Barçu ...,De 28-jarige Vidal kwam drie seizoenen geleden...,28-godišnji Vidal je prije tri sezone prešao i...,21.963695,28.787878
6,6,Sinds hij verhuisde naar de Catalaanse hoofdst...,Vidal je za klub odigrao 49 utakmica otkako se...,Hij heeft voor de club 49 wedstrijden gespeeld...,"Od kada se preselio u glavni grad Katalonije, ...",16.906536,38.975750
7,7,Het protest begon rond 11.00 lokale tijd (UTC+...,Prosvjed je započeo oko 11:00 po lokalnom vrem...,De demonstratie begon rond 11:00 uur (UTC+1) o...,Prosvjedi su počeli oko 11 sati lokalnog vreme...,42.769885,33.235647
8,8,Iets na 11.00 uur begonnen betogers het verkee...,Netom iza 11:00 h prosvjednici su blokirali pr...,Net na 11:00 uur blokkeerden de demonstranten ...,"Nakon 11 sati, prosvjednici su počeli blokirat...",3.037994,6.986768
9,9,Om 11:20 vroeg de politie aan de demonstranten...,Policija je u 11:20 zatražila od prosvjednika ...,Om 11:20 uur heeft de politie de demonstranten...,Policija je u 11:20 zatražila od prosvjednika ...,10.736868,38.797210


In [ ]:
from nltk.translate.meteor_score import single_meteor_score
from nltk import word_tokenize
from tqdm import tqdm
import nltk

# Download NLTK's tokenizer models
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the lists to store METEOR scores
dutch_to_croatian_meteor_scores = []
croatian_to_dutch_meteor_scores = []

# Loop through each row in the DataFrame
for _, row in tqdm(evaluation_df.iterrows(), total=evaluation_df.shape[0]):
    # Tokenize the Dutch and Croatian sentences
    reference_dutch = word_tokenize(row['Dutch'])
    hypothesis_croatian_to_dutch = word_tokenize(row['Translated Dutch'])
    reference_croatian = word_tokenize(row['Croatian'])
    hypothesis_dutch_to_croatian = word_tokenize(row['Translated Croatian'])

    # Calculate the METEOR score for Dutch to Croatian translation
    dutch_to_croatian_score = single_meteor_score(reference_croatian, hypothesis_dutch_to_croatian)
    dutch_to_croatian_meteor_scores.append(dutch_to_croatian_score)

    # Calculate the METEOR score for Croatian to Dutch translation
    croatian_to_dutch_score = single_meteor_score(reference_dutch, hypothesis_croatian_to_dutch)
    croatian_to_dutch_meteor_scores.append(croatian_to_dutch_score)

# Add the METEOR scores to  DataFrame
evaluation_df['Croatian to Dutch METEOR'] = croatian_to_dutch_meteor_scores
evaluation_df['Dutch to Croatian METEOR'] = dutch_to_croatian_meteor_scores

# Display the updated DataFrame
print(evaluation_df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
100%|██████████| 2226/2226 [00:06<00:00, 330.30it/s]

   Unnamed: 0                                              Dutch  \
0           0  Op maandag kondigden wetenschappers van de Sta...   
1           1  Hoofdonderzoekers zeggen dat dit kan leiden to...   
2           2  De JAS 39C Gripen stortte rond 09.30 uur lokal...   
3           3  De piloot werd geïdentificeerd als majoor Dilo...   
4           4  De lokale media meldt dat er tijdens een actie...   

                                            Croatian  \
0  U ponedjeljak su znanstvenici s Medicinskog fa...   
1  Voditelji istraživanja izjavili su da bi ovo o...   
2  JAS 39C Gripen srušio se na pistu i eksplodira...   
3  Pilot je identificiran kao zapovjednik eskadri...   
4  Lokalni mediji izvješćuju da je došlo do prevr...   

                                    Translated Dutch  \
0  Op maandag konden wetenschappers van de School...   
1  De leiders van het onderzoek zeiden dat deze o...   
2  De JAS 39C Gripen raakte op een spoor en explo...   
3  De piloot werd geïdentifice

In [ ]:
print(evaluation_df["Croatian to Dutch METEOR"].mean())

0.4667814568150076


In [ ]:
print(evaluation_df["Dutch to Croatian METEOR"].mean())

0.4150145196415528


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
from evaluate import load
comet_metric = load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
import numpy as np
from tqdm import tqdm

# Function to compute COMET scores in batches
def compute_comet_in_batches(data, batch_size=100):
    scores = []
    for i in tqdm(range(0, len(data['sources']), batch_size)):
        batch = {
            "sources": data['sources'][i:i+batch_size],
            "predictions": data['predictions'][i:i+batch_size],
            "references": data['references'][i:i+batch_size]
        }
        batch_results = comet_metric.compute(predictions=batch["predictions"], references=batch["references"], sources=batch["sources"])
        scores.extend(batch_results["scores"])
    return scores

# Croatian to Dutch COMET scores
data_cro_to_dut = {
    "sources": evaluation_df['Croatian'].tolist(),
    "predictions": evaluation_df['Translated Dutch'].tolist(),
    "references": evaluation_df['Dutch'].tolist()
}

cro_to_dut_scores = compute_comet_in_batches(data_cro_to_dut)
evaluation_df['Croatian to Dutch COMET'] = [round(score, 5) for score in cro_to_dut_scores]

# Dutch to Croatian COMET scores
data_dut_to_cro = {
    "sources": evaluation_df['Dutch'].tolist(),
    "predictions": evaluation_df['Translated Croatian'].tolist(),
    "references": evaluation_df['Croatian'].tolist()
}

dut_to_cro_scores = compute_comet_in_batches(data_dut_to_cro)
evaluation_df['Dutch to Croatian COMET'] = [round(score, 5) for score in dut_to_cro_scores]

# Display the updated DataFrame
print(evaluation_df.head())

  0%|          | 0/23 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  4%|▍         | 1/23 [00:06<02:29,  6.79s/it]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  9%|▊         | 2/23 [00:14<02:34,  7.35s/it]INFO:pytorch_lightning.utilities.rank_zero:GPU available

   Unnamed: 0                                              Dutch  \
0           0  Op maandag kondigden wetenschappers van de Sta...   
1           1  Hoofdonderzoekers zeggen dat dit kan leiden to...   
2           2  De JAS 39C Gripen stortte rond 09.30 uur lokal...   
3           3  De piloot werd geïdentificeerd als majoor Dilo...   
4           4  De lokale media meldt dat er tijdens een actie...   

                                            Croatian  \
0  U ponedjeljak su znanstvenici s Medicinskog fa...   
1  Voditelji istraživanja izjavili su da bi ovo o...   
2  JAS 39C Gripen srušio se na pistu i eksplodira...   
3  Pilot je identificiran kao zapovjednik eskadri...   
4  Lokalni mediji izvješćuju da je došlo do prevr...   

                                    Translated Dutch  \
0  Op maandag konden wetenschappers van de School...   
1  De leiders van het onderzoek zeiden dat deze o...   
2  De JAS 39C Gripen raakte op een spoor en explo...   
3  De piloot werd geïdentifice

In [ ]:
evaluation_df

,Unnamed: 0,Dutch,Croatian,Translated Dutch,Translated Croatian,Croatian to Dutch BLEU,Dutch to Croatian BLEU,Croatian to Dutch METEOR,Dutch to Croatian METEOR,Croatian to Dutch COMET,Dutch to Croatian COMET
0,0,Op maandag kondigden wetenschappers van de Sta...,U ponedjeljak su znanstvenici s Medicinskog fa...,Op maandag konden wetenschappers van de School...,U ponedjeljak su znanstvenici sa Stanford Univ...,8.856387,7.853061,0.463883,0.272280,0.86383,0.71424
1,1,Hoofdonderzoekers zeggen dat dit kan leiden to...,Voditelji istraživanja izjavili su da bi ovo o...,De leiders van het onderzoek zeiden dat deze o...,Glavni istraživači kažu da to može dovesti do ...,26.109285,25.740200,0.431058,0.393498,0.90172,0.90754
2,2,De JAS 39C Gripen stortte rond 09.30 uur lokal...,JAS 39C Gripen srušio se na pistu i eksplodira...,De JAS 39C Gripen raakte op een spoor en explo...,JAS 39C Gripen srušio se oko 09.30 sati lokaln...,15.773903,37.837561,0.590728,0.724382,0.81382,0.91121
3,3,De piloot werd geïdentificeerd als majoor Dilo...,Pilot je identificiran kao zapovjednik eskadri...,De piloot werd geïdentificeerd als commandant ...,Pilot je identificiran kao glavni Dilokrit Pat...,38.163309,44.124845,0.730923,0.777345,0.87555,0.65215
4,4,De lokale media meldt dat er tijdens een actie...,Lokalni mediji izvješćuju da je došlo do prevr...,Lokale media melden dat er een brandweer van d...,Lokalni mediji izvijestili su da je tijekom ak...,6.228497,6.219512,0.397845,0.230480,0.79066,0.57779
...,...,...,...,...,...,...,...,...,...,...,...
2221,212,Mijn naam is Andrea.,Moje ime je Andrea.,Mijn naam is Andrea.,Moje ime je Andrea.,100.000000,100.000000,0.996000,0.996000,0.99040,0.99322
2222,213,Jupiter is een planeet.,Jupiter je planet.,Jupiter is een planeet.,Jupiter je planeta.,100.000000,35.355339,0.996000,0.638889,0.98417,0.97683
2223,214,Venus is een planeet.,Venera je planet.,Venus is een planeet.,Venera je planeta.,100.000000,35.355339,0.996000,0.638889,0.98203,0.97733
2224,215,God is een olifant.,Bog je slon.,God is een olifant.,Bog je slon.,100.000000,100.000000,0.996000,0.992188,0.97846,0.98735


In [ ]:
print(evaluation_df["Croatian to Dutch COMET"].mean())

0.8080632614555256


In [ ]:
print(evaluation_df["Dutch to Croatian COMET"].mean())

0.828725004492363


In [ ]:
evaluation_df.to_csv('/content/drive/MyDrive/m2m100_418M_evaluation_1.csv')